<a href="https://colab.research.google.com/github/OmerDiaaeldin/Machine-Learning/blob/main/FashionMNIST_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

import os

import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [5]:
dir_path = "./fashionmnist/"
if not os.path.exists(dir_path):
  os.makedirs(dir_path)

In [ ]:
train_data = datasets.FashionMNIST(
    root = dir_path,
    train = True,
    download=True,
    transform = ToTensor(),
    target_transform=None,
)

test_data = datasets.FashionMNIST(
    root = dir_path,
    train = False,
    download=True,
    transform = ToTensor(),
)

In [7]:
image, label = train_data[0]

In [ ]:
train_data.classes

In [ ]:
plt.imshow(image.squeeze(), cmap = plt.get_cmap("gray"))

In [10]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_dataloader = DataLoader(
    train_data,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

test_dataloader = DataLoader(
    test_data,
    batch_size = BATCH_SIZE,
)

In [11]:
from timeit import default_timer as timer

def print_time(start,end):
  total_time = end - start
  print(f"Train time {total_time:.3f} seconds")
  return total_time

In [107]:
from tqdm.auto import tqdm
class model(nn.Module):
  def __init__(self, input_shape, output_shape):
    super().__init__()
    self.train_loss_history = []
    self.test_loss_history = []
    self.train_accuracy_history = []
    self.test_accuracy_history = []
    self.in_channels = input_shape[0]
    self.height = input_shape[1]
    self.width = input_shape[2]
    self.output_shape = output_shape
    self.block1 = nn.Sequential(
        nn.Conv2d(in_channels = self.in_channels, 
                  out_channels = 32,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1,
                  ),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.block2 = nn.Sequential(
        nn.Conv2d(in_channels = 32, 
                  out_channels = 64,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1,
                  ),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Dropout(0.2),
        nn.Linear(in_features=64*7*7, out_features=10),
    )

    self.loss_fn = nn.CrossEntropyLoss()
    self.optimizer = torch.optim.SGD(self.parameters(), lr = 0.1)

  def forward(self, x:torch.Tensor):
    x = self.block1(x)
    x = self.block2(x)
    x = self.classifier(x)
    return x

  def accuracy_fn(self, y_pred, y_true):
    return torch.eq(y_pred.argmax(dim=1), y_true).sum().item()*100/len(y_true)

  def fit(self, epochs, train_data_loader: torch.utils.data.DataLoader, test_data_loader: torch.utils.data.DataLoader):
    
    start_time = timer()
    for epoch in tqdm(range(epochs)):
      
      train_loss = 0
      train_acc = 0
      
      for batch, (X,y) in enumerate(train_data_loader):

        X,y = X.to(device), y.to(device)
        self.train()
        y_pred = self(X)

        loss = self.loss_fn(y_pred, y)
        train_loss += loss
        train_acc += self.accuracy_fn(y_pred,y)

        self.optimizer.zero_grad()

        loss.backward()

        self.optimizer.step()

        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/{len(train_dataloader.dataset)} samples")

      train_loss /= len(train_data_loader)
      train_acc /= len(train_data_loader)

      test_loss, test_acc = 0, 0 
      self.eval()
      with torch.inference_mode():
        for (X, y) in test_data_loader:
          X,y = X.to(device), y.to(device)
          test_pred = self(X)
          test_loss += self.loss_fn(test_pred, y)
          test_acc += self.accuracy_fn(test_pred,y)
        test_loss /= len(test_data_loader)
        test_acc /= len(test_data_loader)
      self.train_loss_history.append(train_loss.to('cpu').item())
      self.test_loss_history.append(test_loss.to('cpu').item())
      self.train_accuracy_history.append(train_acc)
      self.test_accuracy_history.append(test_acc)
      end_time = timer()
      print(f"\nTrain loss: {train_loss:.5f}, Train acc: {train_acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%\n")
      print_time(start_time,end_time)
    return [self.train_loss_history, self.test_loss_history, self.train_accuracy_history, self.test_accuracy_history]

In [108]:
model0 = model([1,28,28],10).to(device)

In [ ]:
[train_loss, test_loss, train_acc, test_acc] = model0.fit(10,train_dataloader, test_dataloader)

In [ ]:
from random import randrange
choice = randrange(len(test_data))
with torch.inference_mode():
  image = test_data[choice][0].unsqueeze(dim=0)
  label = test_data[choice][1]

  plt.imshow(image.squeeze(), cmap = plt.get_cmap('gray'))

  image = image.to(device)
  logits = model0(image)
  pred = torch.argmax(logits).item()
  print(f"pred: {pred},label: {label}")
  print(f"class: {train_data.classes[pred]}")

In [ ]:
plt.plot(train_loss, label='train')
plt.plot(test_loss, label='test')
plt.xlabel('iterations')
plt.ylabel('loss')
plt.title("loss vs iterations")
plt.legend()
plt.show()

In [ ]:
plt.plot(train_acc, label = 'train')
plt.plot(test_acc, label='test')
plt.xlabel('iterations')
plt.ylabel('accuracy')
plt.title("accuracy vs iterations")
plt.legend()
plt.show()